In [119]:
import numpy as np
from scipy.sparse.linalg import eigsh
from itertools import product
from collections import defaultdict
import time
import csv
import functions as func
import hamiltonianGenerator as hG
import pandas as pd
from functools import reduce

# Applying the delta function operation.

In [2]:
sites = 3
states = 2

In [76]:
matrix = np.zeros((9,9))
matrix[1,2] = 1
matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [80]:
for p in range(0,9):
    for q in range(0,9):
        for r in range(0,9):
            matrix[r,q] = p + q*10 + r*100

matrix

TypeError: list indices must be integers or slices, not tuple

In [110]:
def generate_matrix(site, state):
    states = ["".join(map(str, digits)) for digits in product(range(state), repeat=site)]
    matrix = []

    print(states)

    for row in states:
        row_entries = [f"{row}|{col}" for col in states]
        matrix.append(row_entries)

    return matrix


def print_matrix(matrix):
    for row in matrix:
        print(" ".join(row))

In [ ]:
# Example usage
site = 3
state = 3
matrix_3 = generate_matrix(site, state)
print_matrix(matrix_3)


['000', '001', '010', '011', '100', '101', '110', '111']
000|000 000|001 000|010 000|011 000|100 000|101 000|110 000|111
001|000 001|001 001|010 001|011 001|100 001|101 001|110 001|111
010|000 010|001 010|010 010|011 010|100 010|101 010|110 010|111
011|000 011|001 011|010 011|011 011|100 011|101 011|110 011|111
100|000 100|001 100|010 100|011 100|100 100|101 100|110 100|111
101|000 101|001 101|010 101|011 101|100 101|101 101|110 101|111
110|000 110|001 110|010 110|011 110|100 110|101 110|110 110|111
111|000 111|001 111|010 111|011 111|100 111|101 111|110 111|111


In [ ]:
def encode_tuple(digits):
    """Efficiently convert a tuple of digits to a single integer (e.g., (0,1,0) → 10)."""
    result = 0
    for d in digits:
        result = result * 10 + d
    return result

def generate_matrix(site, state):
    # Generate all basis states as tuples of digits
    states = list(product(range(state), repeat=site))
    

    # Precompute integer encoding for each state
    encoded = [encode_tuple(s) for s in states]

    # Compute digit shift for concatenation
    digit_scale = 10 ** site

    # Build matrix using pure integer math
    matrix = np.array([[row * digit_scale + col for col in encoded] for row in encoded])
    return matrix

In [117]:
matrix_3 = generate_matrix(site, state)
print(matrix_3)

[[     0      1     10     11    100    101    110    111]
 [  1000   1001   1010   1011   1100   1101   1110   1111]
 [ 10000  10001  10010  10011  10100  10101  10110  10111]
 [ 11000  11001  11010  11011  11100  11101  11110  11111]
 [100000 100001 100010 100011 100100 100101 100110 100111]
 [101000 101001 101010 101011 101100 101101 101110 101111]
 [110000 110001 110010 110011 110100 110101 110110 110111]
 [111000 111001 111010 111011 111100 111101 111110 111111]]


In [ ]:
import numpy as np

# Sample tensor of shape (2, 3, 8, 8)
T = np.random.rand(2, 3, 8, 8)

# Use broadcasting to compare across n
comparison = T[0] != T[1]  # shape: (3, 8, 8), boolean

# Convert True/False to 1/0
comparison_int = comparison.astype(int)  # shape: (3, 8, 8)

In [118]:
T = np.arange(3**3).reshape(3, 3, 3)
print("Original Tensor (flattened):")
print(T.flatten())
print(T)

for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 4):
            T[i-1,j-1,k-1] = 100*i+10*j+k

print(T)

# Contract i with k: enforce i == k
T_masked = np.einsum('ij,kl,ijkl->ijkl', np.eye(3), np.eye(3), T)

Original Tensor (flattened):
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]]
[[[111 112 113]
  [121 122 123]
  [131 132 133]]

 [[211 212 213]
  [221 222 223]
  [231 232 233]]

 [[311 312 313]
  [321 322 323]
  [331 332 333]]]


ValueError: einstein sum subscripts string contains too many subscripts for operand 2

In [123]:
T = np.arange(3**2).reshape(3, 3)
print("Original Tensor (flattened):")
print(T.flatten())

for i in range(1, 4):
    for j in range(1, 4):
        T[i-1,j-1] = 10*i+1*j

matrices = [T, np.eye(3), np.eye(3)]
T_new = reduce(np.kron, matrices)

print(matrices)
print(T)
print(T_new.shape)

Original Tensor (flattened):
[0 1 2 3 4 5 6 7 8]
[array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])]
[[11 12 13]
 [21 22 23]
 [31 32 33]]
(27, 27)


In [143]:
matrices = [T, np.eye(3), np.eye(3)]
matrices[0] = T
print(matrices)

print(np.eye(3).shape[0])

[array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])]
3


In [157]:
def compute_kinetic_matrix_individual_sites(K: np.ndarray, site: int, state: int)-> np.ndarray:

    dim = K.shape[0]
    site = site -1

    matrices = [0]*state
    print(matrices)
    matrices[site] = K

    for i in range(state):

        if i == site:
            matrices[i] = K
        else:
            matrices[i] = np.eye(state)

    K = reduce(np.kron, matrices)

    return K

def compute_kinetic_matrix(K: np.ndarray, site: int, state: int)-> np.ndarray:

    K_final = np.zeros(site**state)

    for i in range(site):
        K_final = K_final + compute_kinetic_matrix_individual_sites(K, i, state)

    return K_final

In [159]:
site = 3
state = 3
K = np.eye(3)


print(compute_kinetic_matrix_individual_sites(K, site, state))
print(compute_kinetic_matrix(K, site, state))

[0, 0, 0]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 

In [103]:
n = 3  # basis size

h = np.eye(n, n)  # one-body operator h_{p p'}
delta = np.eye(n)

# Broadcasted construction
H = h[:, None, None, None, :, None, None, None] \
  * delta[None, :, None, None, None, :, None, None] \
  * delta[None, None, :, None, None, None, :, None]

H = np.einsum('pP,qQ,rR->pP', h, delta, delta)

H

array([[9., 0., 0.],
       [0., 9., 0.],
       [0., 0., 9.]])

# Converting from m to p.

## Free one body system. (Work in progress)

In [27]:

# Original matrix A
A = np.array([
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10]
])
print(A)
# Index map: new_index[i] = index of row i in the new basis
index_map = [0, 3, 1, 2]  # Example: remap row/col 1 → 3, 2 → 1, 3 → 2

# Permute rows and columns accordingly
A_new = A[np.ix_(index_map, index_map)]
A_new = A_new.T[np.ix_(index_map, index_map)]

print(np.ix_(index_map, index_map))

print(A_new)

P = np.eye(len(index_map))[index_map]
A_new2 = P.T @ A @ P

print(A_new2)

[[ 1  2  3  4]
 [ 2  5  6  7]
 [ 3  6  8  9]
 [ 4  7  9 10]]
(array([[0],
       [3],
       [1],
       [2]]), array([[0, 3, 1, 2]]))
[[ 1  3  4  2]
 [ 3  8  9  6]
 [ 4  9 10  7]
 [ 2  6  7  5]]
[[ 1.  3.  4.  2.]
 [ 3.  8.  9.  6.]
 [ 4.  9. 10.  7.]
 [ 2.  6.  7.  5.]]


In [8]:
def free_one_body(max_m: int)-> np.array:

    p_max = 2*max_m + 1
    p_vect = np.zeros(p_max)

    m = 0

    for i in range(1, max_m+1):

        p_vect[i + m] = i**2
        p_vect[i + 1 + m] = (-i)**2
        m+=1
    
    print(p_vect)

    return  np.diag(p_vect)

K_new = free_one_body(5)

K_new

[ 0.  1.  1.  4.  4.  9.  9. 16. 16. 25. 25.]


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 25.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 25.]])

In [9]:
df = pd.read_csv(
    "/Users/gilfrim/Desktop/QuantumChemistryCoop/Main-CC-files/matrix_elements_K.csv",
    skiprows=1, header=0
)

# Clean column names
df.columns = [col.strip() for col in df.columns]

# Convert columns safely
df["m1"] = df["m1"].astype(int)
df["m2"] = df["m2"].astype(int)
df["<m1|K|m2>"] = df["<m1|K|m2>"].astype(float)

# Matrix size and init
size = max(df["m1"].max(), df["m2"].max()) + 1
K = np.zeros((size, size))

m = (size - 1) / 2

# Fill matrix
for _, row in df.iterrows():

    i = int(row["m1"])
    j = int(row["m2"])
    val = float(row["<m1|K|m2>"])
    K[i, j] = val
    K[j, i] = val

    
# Show matrix
print(pd.DataFrame(K))

      0     1    2    3    4    5    6    7    8     9     10
0   25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
1    0.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
2    0.0   0.0  9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
3    0.0   0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
4    0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0
5    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
6    0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   0.0   0.0
7    0.0   0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0   0.0   0.0
8    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0   0.0   0.0
9    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  16.0   0.0
10   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  25.0


In [10]:
# p_vect = np.zeros(11)
p_vect = np.array([0,  -1, 1,  -2, 2,  -3, 3,  -4, 4,  -5, 5])
m=0
vec1 = np.zeros(11)

for i in range(11):
    vec1[i] = int(func.m_to_p(p_vect[i]))

print(p_vect)
print(vec1)

[ 0 -1  1 -2  2 -3  3 -4  4 -5  5]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


## Free one body system. (Final Function)

In [ ]:
def create_index_map(total_num_states: int)->np.array:

    index_map = np.arange(-total_num_states,total_num_states+1)

    vectorised_m_to_p = np.vectorize(func.m_to_p)
    index_map = vectorised_m_to_p(index_map)

    return index_map

print(create_index_map(9))

def basis_m_to_p_matrix_conversion(matrix: np.ndarray)->np.ndarray:

    dim = matrix.shape[0]
    index_map = create_index_map((dim-1)//2)

    matrix = matrix[np.ix_(index_map, index_map)]

    return matrix

print(pd.DataFrame(basis_m_to_p_matrix_conversion(K)))

[17 15 13 11  9  7  5  3  1  0  2  4  6  8 10 12 14 16 18]
     0    1     2    3    4     5    6     7    8    9     10
0   9.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
1   0.0  1.0   0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
2   0.0  0.0  25.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
3   0.0  0.0   0.0  4.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
4   0.0  0.0   0.0  0.0  4.0   0.0  0.0   0.0  0.0  0.0   0.0
5   0.0  0.0   0.0  0.0  0.0  16.0  0.0   0.0  0.0  0.0   0.0
6   0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
7   0.0  0.0   0.0  0.0  0.0   0.0  0.0  16.0  0.0  0.0   0.0
8   0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  9.0  0.0   0.0
9   0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  0.0  1.0   0.0
10  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  25.0


In [ ]:
df = pd.read_csv(
    "/Users/gilfrim/Desktop/QuantumChemistryCoop/Main-CC-files/matrix_elements_K.csv",
    skiprows=1, header=0
)

# Clean column names
df.columns = [col.strip() for col in df.columns]

# Convert columns safely
df["m1"] = df["m1"].astype(int)
df["m2"] = df["m2"].astype(int)
df["<m1|K|m2>"] = df["<m1|K|m2>"].astype(float)

# Matrix size and init
size = max(df["m1"].max(), df["m2"].max()) + 1
K = np.zeros((size, size))

m = (size - 1) / 2

# Fill matrix
for _, row in df.iterrows():

    i = int(row["m1"])
    j = int(row["m2"])
    val = float(row["<m1|K|m2>"])
    K[i, j] = val
    K[j, i] = val

    
# Show matrix
print(pd.DataFrame(K))

print(pd.DataFrame(basis_m_to_p_matrix_conversion(K)))

      0     1    2    3    4    5    6    7    8     9     10
0   25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
1    0.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
2    0.0   0.0  9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
3    0.0   0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
4    0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0
5    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
6    0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   0.0   0.0
7    0.0   0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0   0.0   0.0
8    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0   0.0   0.0
9    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  16.0   0.0
10   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  25.0
     0    1     2    3    4     5    6     7    8    9     10
0   9.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
1   0.0  1.0   0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
2   0.0  0.0  25.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0   0.0
3   0.0 

## Coplaner two body system. (Work in progress)

### Learning einsum

1. Inner product of vectors

'i,i->': sum over index i (vector inner product)

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

np.dot(a, b)         # → 32
np.einsum('i,i->', a, b)  # → 32

32

2. Matrix multiplication

'ik,kj->ij': sum over index k (standard matrix multiplication)

In [ ]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[2, 0], [1, 2]])

np.matmul(A, B)
np.einsum('ik,kj->ij', A, B)

array([[ 4,  4],
       [10,  8]])

3. Trace of a matrix

'ii->': sum diagonal elements (same index repeated = sum)

In [ ]:
a = np.array([1, 2])
b = np.array([3, 4, 5])

np.einsum('i,j->ij', a, b)
# Result: 2x3 matrix of a[i] * b[j]

array([[ 3,  4,  5],
       [ 6,  8, 10]])

4. Tensor contraction

Contracts the last index of T with w (like applying weights)

In [ ]:
T = np.random.rand(2, 3, 4)
w = np.random.rand(4)

np.einsum('ijk,k->ij', T, w)

array([[0.93882073, 0.96995704, 1.08514116],
       [0.734377  , 0.76955526, 0.76790332]])

5. Outer product

In [ ]:
a = np.array([1, 2])
b = np.array([3, 4, 5])

np.einsum('i,j->ij', a, b)
# Result: 2x3 matrix of a[i] * b[j]

array([[ 3,  4,  5],
       [ 6,  8, 10]])

###

In [ ]:

T = np.arange(3**2).reshape(3, 3)
print("Original Tensor (flattened):")
print(T.flatten())

for i in range(1, 4):
    for j in range(1, 4):
        T[i-1,j-1] = 100*i+10*j

index_map = [1, 0, 2]  # Permute axis 0
index_maps = []
for i in range(2):
    index_maps.append(index_map)

print(index_maps)
# Unpack index_maps into np.ix_
T_permuted = T[np.ix_(*index_maps)]

P = np.eye(len(index_map))[index_map]
T_einsum = np.einsum('ai,bj,ij->ab', P, P, T)

print(P.shape)
print(T.shape)

T_change_of_bais = P.T @ T @ P

print("Original slice T[0, 1, :, :]:")
print(T)

print("\nPermuted slice T_permuted[0, 1, :, :]:")
print(T_permuted)
print(T_einsum)
print(T_change_of_bais)


Original Tensor (flattened):
[0 1 2 3 4 5 6 7 8]
[[1, 0, 2], [1, 0, 2]]
(3, 3)
(3, 3)
Original slice T[0, 1, :, :]:
[[110 120 130]
 [210 220 230]
 [310 320 330]]

Permuted slice T_permuted[0, 1, :, :]:
[[220 210 230]
 [120 110 130]
 [320 310 330]]
[[220. 210. 230.]
 [120. 110. 130.]
 [320. 310. 330.]]
[[220. 210. 230.]
 [120. 110. 130.]
 [320. 310. 330.]]


In [66]:

T = np.arange(3**3).reshape(3, 3, 3)
print("Original Tensor (flattened):")
print(T.flatten())

for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 4):
            T[i-1,j-1,k-1] = 100*i+10*j+k

index_map = [1, 0, 2]  # Permute axis 0
index_maps = []
for i in range(3):
    index_maps.append(index_map)

print(index_maps)
# Unpack index_maps into np.ix_
T_permuted = T[np.ix_(*index_maps)]

print("Original slice T[0, 1, :, :]:")
print(T[0, :, :])

print("\nPermuted slice T_permuted[0, 1, :, :]:")
print(T_permuted[0, :, :])

print(index_maps)

Original Tensor (flattened):
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]
[[1, 0, 2], [1, 0, 2], [1, 0, 2]]
Original slice T[0, 1, :, :]:
[[111 112 113]
 [121 122 123]
 [131 132 133]]

Permuted slice T_permuted[0, 1, :, :]:
[[222 221 223]
 [212 211 213]
 [232 231 233]]
[[1, 0, 2], [1, 0, 2], [1, 0, 2]]


In [64]:
T

array([[[111, 112, 113],
        [121, 122, 123],
        [131, 132, 133]],

       [[211, 212, 213],
        [221, 222, 223],
        [231, 232, 233]],

       [[311, 312, 313],
        [321, 322, 323],
        [331, 332, 333]]])

## Coplaner two body system. (Final Function)

In [68]:
def create_index_map(total_num_states: int)->np.array:

    index_map = np.arange(-total_num_states,total_num_states+1)

    vectorised_m_to_p = np.vectorize(func.m_to_p)
    index_map = vectorised_m_to_p(index_map)

    return index_map

print(create_index_map(1))

def basis_m_to_p_matrix_conversion(matrix: np.ndarray)->np.ndarray:

    dim = matrix.shape[0]
    index_map = create_index_map((dim-1)//2)

    index_maps = []
    for i in range(dim):
        index_maps.append(index_map)

    print(index_maps)

    matrix = matrix[np.ix_(*index_maps)]

    return matrix

print(basis_m_to_p_matrix_conversion(T))

[1 0 2]
[array([1, 0, 2]), array([1, 0, 2]), array([1, 0, 2])]
[[[222 221 223]
  [212 211 213]
  [232 231 233]]

 [[122 121 123]
  [112 111 113]
  [132 131 133]]

 [[322 321 323]
  [312 311 313]
  [332 331 333]]]


# Creating Sheres matrix with numpy.

$$
V = 0.75 \sum_{m_1, m_2} \left( |m_1, m_2\rangle \langle m_1 + 1, m_2 + 1| + |m_1, m_2\rangle \langle m_1 - 1, m_2 - 1| \right) \\
\quad -\ 0.25 \sum_{m_1, m_2} \left( |m_1, m_2\rangle \langle m_1 - 1, m_2 + 1| + |m_1, m_2\rangle \langle m_1 + 1, m_2 - 1| \right)
$$

In [ ]:
import numpy as np
import hamiltonianGenerator as hg
import pandas as pd

np.set_printoptions(suppress = True, linewidth = 10000, threshold = 1000000, precision = 6)

# Script for diagonalizing the Hamiltonian for 2 dipolar rotors oriented along the x direction. 
# m_max = 5 is sufficient for convergence when studying ground state properties. 
m_max = 5
d = 2*m_max + 1
K_1 = np.zeros((d,d))
for i in range(d):
    K_1[i,i] = hg.free_one_body(i, i, m_max)

K_2 = np.kron(K_1, np.eye(d)) + np.kron(np.eye(d), K_1)

# Interaction for a coplanar chain is proportional to (yiyj + 2xixj)/r^3. If sites are not coplanar, 
# the interaction will be some other linear combination of xixj, yiyj, (xiyj + xjyi). The coefficients 
# will, in general, depend on the angle as well as the length of the vector connecting the sites. 
V2 = np.zeros((d**2, d**2))

for i in range(d):
    for j in range(d):
        for k in range(d):
            for l in range(d):
                V2[i*d + j,k*d + l] = hg.interaction_yiyj(i, j, k, l) + 2*hg.interaction_xixj(i, j, k, l)
'''
num_pts = 20
E0_array = np.zeros((num_pts, 2))
for i in range(num_pts):
    g = (i+1.0)/num_pts
    H_2 = K_2 + g * V_2
    evals, evecs = np.linalg.eigh(H_2)
    E0 = evals[0]
    E0_array[i, 0] = g
    E0_array[i, 1] = E0

np.savetxt("E0_N2_ED.csv", E0_array, delimiter=",", header="g, E0")'''


print(pd.DataFrame(V2))

     0    1    2    3    4    5    6    7    8    9    ...  111  112  113  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
116  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
117  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
118  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
119  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
120  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     114  115  116  117  118  119  120  
0    0.0  0.0  0.0  0.0  0.0  0.0 